In [7]:
import io
import json
import os
import sys
import random
import csv
import subprocess
import numpy as np
import pandas as pd
import tifffile as tiff
from PIL import Image
from math import sqrt, exp, log
from matplotlib import cm
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
import re
import shutil
from flir_image_extractor import FlirImageExtractor

In [12]:
os.chdir(sys.path[0])
rgbt_jpg_source_dir = os.path.join(
    'raw_data_source', 'RGBT_jpg')
t_npy_dest_dir = os.path.join(
    'raw_data_source', 'T_npy')
rgb_npy_dest_dir = os.path.join(
    'raw_data_source', 'RGB_npy')

rgbt_jpg_file_list = sorted([os.path.join(
    rgbt_jpg_source_dir, n) for n in os.listdir(rgbt_jpg_source_dir) if not n.startswith('.') and n.endswith('.jpg')])

In [ ]:
fie = FlirImageExtractor(exiftool_path='exiftool')

raw_dataset = [] 

for rgbt_file in tqdm(rgbt_jpg_file_list):
    meta = None

    try:
        fie.process_image(rgbt_file)
    except Exception as e:
        print(f"An error occurred calling fie.process_image(): {e}")
    
    try:
        meta = fie.get_metadata()
    except Exception as e:
        print(f"An error occurred calling fie.get_metadata(): {e}")
    
    try:
        thermal_channel = fie.get_thermal_np()
        min_temp = np.min(thermal_channel)
        max_temp = np.max(thermal_channel)
        delta_temp = abs(max_temp-min_temp)
        thermal_channel_norm = (thermal_channel.astype(np.float32) - min_temp)/delta_temp
        cmap = plt.colormaps.get_cmap('rainbow')  # or 'nipy_spectral'
        thermal_channel_as_rgb = cmap(thermal_channel_norm)[:, :, :3]
        dest_npy_file = os.path.join(t_npy_dest_dir, f'{Path(rgbt_file).stem}.npy')
        plt.imshow(thermal_channel_as_rgb)
        plt.title(f'{Path(dest_npy_file).stem}')
        plt.show()
        #np.save(dest_npy_file, thermal_channel)
        meta["t_numpy_file"] = dest_npy_file
    except Exception as e:
        print(f"An error occurred calling fie.get_thermal_np(): {e}")
        meta["t_numpy_file"] = None
    
    try:
        rgb_channels = fie.get_rgb_np()
        dest_npy_file = os.path.join(rgb_npy_dest_dir, f'{Path(rgbt_file).stem}.npy')
        # np.save(dest_npy_file, rgb_channels)
        meta["rgb_numpy_file"] = dest_npy_file
    except Exception as e:
        print(f"An error occurred calling fie.get_rgb_np(): {e}")
        meta["rgb_numpy_file"] = None

    if meta is not None:
        # json_string = json.dumps(meta, indent=4).replace('null', 'None')
        raw_dataset.append(meta)

# df = pd.DataFrame(raw_dataset)

# Save the DataFrame to a CSV file
# df.to_csv('raw_data_source.csv', index=False)